In [ ]:
from sklearn.neural_network import MLPClassifier
import os
import cv2
from skimage.feature import hog
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from tqdm import tqdm_notebook
from sklearn.externals import joblib

In [ ]:
def eks_hog(gambar):
    eks_ciri, hog_image = hog(gambar, block_norm='L2-Hys',orientations=8, cells_per_block=(6, 6), pixels_per_cell=(6,6), visualize=True, multichannel=True)

    return eks_ciri

In [ ]:
#direct = 'G:/Program TA/data coba/train'
direct = 'G:/Program TA/data scan/train1'
melihat_isi_direct = os.listdir(direct)
print('Total gambar kayu:',len(melihat_isi_direct))
#print('Isi File = ', melihat_isi_direct)

In [ ]:
def labelling(nama):
    if 'A' in nama:
        label = 0
    if 'B' in nama:
        label = 1
    if 'C' in nama:
        label = 2
    if 'D' in nama:
        label = 3
    if 'E' in nama:
        label = 4
        
    return label

In [ ]:
def preprocessing(path):

    train_label = []
    train_ekstraksi = []
    melihat_isi_direct = os.listdir(path)

    for gambar in tqdm_notebook(melihat_isi_direct, desc="Proses"):

        label = labelling(gambar)
        img = cv2.imread(os.path.join(path, gambar))
        res = cv2.resize(img, (60, 60))
        hasil = eks_hog(res)

        train_label.append(label)
        train_ekstraksi.append(hasil)
        
    return  np.array(train_ekstraksi), np.array(train_label)

In [ ]:
ekstraksi_ciri_data_latih, label_data_latih = preprocessing(direct)

print('dimensi ekstraksi ciri =', ekstraksi_ciri_data_latih.shape)
print('dimensi label = ', label_data_latih.shape)

X,y = ekstraksi_ciri_data_latih, label_data_latih

In [ ]:
path_data_validasi= 'G:/Program TA/data scan/validate30%'

In [ ]:
ekstraksi_ciri_data_uji, label_data_uji = preprocessing(path_data_validasi)

print('dimensi ekstraksi ciri =', ekstraksi_ciri_data_uji.shape)
print('dimensi label = ', label_data_uji.shape)

Q,w = ekstraksi_ciri_data_uji, label_data_uji

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=80, alpha=1e-4,
                    solver='sgd', verbose=True, tol=1e-6, random_state=0,
                    learning_rate_init=.1)
mlp.fit(X, y)
print("Training set score: %f" % mlp.score(X, y))
#print("Test set score: %f" % mlp.score(Q, w))

In [ ]:
print("Test set score: %f" % mlp.score(Q, w))

In [ ]:
y_prediction = mlp.predict(Q)

In [ ]:
y_true, y_pred = w , mlp.predict(Q)

from sklearn.metrics import classification_report,confusion_matrix
print()
print(confusion_matrix(w,y_pred))
print('Results on the test set:')

print(classification_report(y_true, y_pred))

In [ ]:
# untuk mengecek apakah tidak overfitting dibandingkan dengan data y_test (data uji label)

print(y_prediction)
print('\n')
print(w)

In [ ]:
print('akurasi =', accuracy_score(w, y_prediction)*100)

In [ ]:
joblib.dump(mlp, 'model/saved_model.pkl')

In [ ]:
y_prediction = mlp.predict(Q)
predict_train = mlp.predict(X)

from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y,predict_train))
print(classification_report(y,predict_train))

print('===========================================================================================')

In [ ]:
#data split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=0)

print('dimensi X_train=', X_train.shape)
print('dimensi X_test=', X_test.shape)
print('dimensi y_train=', y_train.shape)
print('dimensi y_test=', y_test.shape)
print('\n')

print(y_test)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=50, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-6, random_state=0,
                    learning_rate_init=.1)


mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))


In [ ]:
y_prediction = mlp.predict(X_test)
predict_train = mlp.predict(X_train)

from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

In [ ]:
y_prediction = mlp.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_prediction))
print(classification_report(y_test,y_prediction))